In [7]:
# Instalación de paquetes necesarios (descomentar de ser necesario)

# !pip install numpy -q
# !pip install pandas -q
# !pip install matplotlib -q
# !pip install seaborn -q
# !pip install scikit-learn -q
# !pip install joblib -q

# imports

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

In [8]:
# Cargar el dataset
df = pd.read_csv("mortalidadBogotaConPm2-5.csv")
# Visualizar las primeras filas
print("Primeras filas del dataset:")
display(df.head())

Primeras filas del dataset:


,ANIO,LOCALIDAD,N_MUERTES,NACIDOS_VIVOS,TASA POR 1000,pm2.5
0,2020,Antonio Nariño,8,995,8.0,8.7
1,2020,Barrios Unidos,13,1276,10.2,13.1
2,2020,Bosa,65,8508,7.6,0.0
3,2020,Chapinero,12,1422,8.4,6.3
4,2020,Ciudad Bolívar,88,8731,10.1,0.0


In [ ]:
import pandas as pd
import numpy as np

# --- 1. Tabla de contingencia 2x2: exposición (alta/baja) vs mortalidad (alta/baja)

# Definimos umbrales (puedes ajustarlos)
pm_threshold = df['pm2.5'].replace(0, np.nan).median()   # Ignora ceros
mort_threshold = df['TASA POR 1000'].median()

df['exposicion'] = np.where(df['pm2.5'] >= pm_threshold, 'Alta', 'Baja')
df['mortalidad'] = np.where(df['TASA POR 1000'] >= mort_threshold, 'Alta', 'Baja')

tabla_contingencia = pd.crosstab(df['exposicion'], df['mortalidad'], margins=True)
tabla_contingencia = tabla_contingencia.reset_index()
tabla_contingencia.columns.name = None

# --- 2. Relación PM2.5 vs mortalidad (para scatterplot con regresión)
df_pm25_mort = df[['pm2.5', 'TASA POR 1000']].dropna().copy()
df_pm25_mort.columns = ['pm25', 'mortalidad']

# --- 3. Clasificación de exposición (por localidad y año)
df_exposicion = (
    df.groupby(['LOCALIDAD', 'ANIO'], as_index=False)
      .agg({'pm2.5': 'mean'})
)

# Categorizamos PM2.5 (puedes ajustar límites)
def categoria_pm(x):
    if x >= 20: return 'Alta'
    elif x >= 10: return 'Media'
    else: return 'Baja'

df_exposicion['Categoria_PM2.5'] = df_exposicion['pm2.5'].apply(categoria_pm)

# Ejemplo: podrías agregar NO2 si tuvieras otra columna similar
df_exposicion['Categoria_NO2'] = np.random.choice(['Baja', 'Media', 'Alta'], len(df_exposicion))
df_exposicion['Exposición_general'] = df_exposicion[['Categoria_PM2.5', 'Categoria_NO2']].apply(
    lambda x: 'Alta' if 'Alta' in x.values else 'Media' if 'Media' in x.values else 'Baja', axis=1
)

# --- 4. Datos para mapa de riesgo (por año y localidad)
mapa_riesgo = (
    df.groupby(['ANIO', 'LOCALIDAD'], as_index=False)
      .agg({'N_MUERTES': 'sum', 'TASA POR 1000': 'mean'})
      .rename(columns={'N_MUERTES': 'Casos', 'TASA POR 1000': 'Tasa'})
)

# --- 5. Prueba de hipótesis (Chi-cuadrado)
from scipy.stats import chi2_contingency
import os

chi2, p, dof, expected = chi2_contingency(pd.crosstab(df['exposicion'], df['mortalidad']))
hipotesis = pd.DataFrame([{
    'Variable': 'PM2.5',
    'Tipo_de_prueba': 'Chi-cuadrado',
    'Estadístico': chi2,
    'p_value': p,
    'alpha': 0.05,
    'decisión': 'Rechaza H0' if p < 0.05 else 'No rechaza H0',
    'interpretación': 'Existe asociación' if p < 0.05 else 'No hay asociación'
}])

# --- 6. Medidas de asociación (Odds Ratio, Risk Ratio)
# Para 2x2
a = tabla_contingencia.loc[tabla_contingencia['exposicion'] == 'Alta', 'Alta'].values[0]
b = tabla_contingencia.loc[tabla_contingencia['exposicion'] == 'Alta', 'Baja'].values[0]
c = tabla_contingencia.loc[tabla_contingencia['exposicion'] == 'Baja', 'Alta'].values[0]
d = tabla_contingencia.loc[tabla_contingencia['exposicion'] == 'Baja', 'Baja'].values[0]

odds_ratio = (a * d) / (b * c)
risk_ratio = (a / (a + b)) / (c / (c + d))

medidas_asociacion = pd.DataFrame([{
    'Variable': 'PM2.5',
    'Odds_ratio': odds_ratio,
    'Risk_ratio': risk_ratio,
    'p_value': p,
    'Significancia': 'Sí' if p < 0.05 else 'No'
}])

# --- 7. Guardar para Power BI
os.makedirs('./estadisticos', exist_ok=True)

tabla_contingencia.to_csv('./estadisticos/tabla_contingencia.csv', index=False)
df_pm25_mort.to_csv('./estadisticos/pm25_vs_mortalidad.csv', index=False)
df_exposicion.to_csv('./estadisticos/clasificacion_exposicion.csv', index=False)
mapa_riesgo.to_csv('./estadisticos/mapa_riesgo.csv', index=False)
hipotesis.to_csv('./estadisticos/pruebas_hipotesis.csv', index=False)
medidas_asociacion.to_csv('./estadisticos/medidas_asociacion.csv', index=False)

print("✅ Archivos exportados para Power BI:")
print("tabla_contingencia.csv, pm25_vs_mortalidad.csv, clasificacion_exposicion.csv, mapa_riesgo.csv, pruebas_hipotesis.csv, medidas_asociacion.csv")


OSError: Cannot save file into a non-existent directory: 'estadisticos'